Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Десяткина Лидия"
COLLABORATORS = ""

---

In [2]:
import numpy as np

from numpy.testing import assert_allclose


### Часть I. Постройте отражение Хаусхолдера для вектора.

Дан вектор $\mathbf{x}$ и плоскость, заданная вектором нормали $\mathbf{u}$. Преобразование Хаусхолдера отражает $\mathbf{x}$ относительно плоскости.

Матрица преобразований Хаусхолдера:
$$ \mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T $$

Если даны два вектора $\mathbf{x}$ и $\mathbf{y}$ одинаковой длины, поворот, преобразующий $\mathbf{x}$ в $\mathbf{y}$ называется преобразованием Хаусхолдера с
$$ \mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{\left|\mathbf{x} - \mathbf{y}\right|} $$

Напишите функцию, преобразующую заданный вектор $\mathbf{x} = (x_1, \dots, x_n)$ в $\mathbf{y} = (\left|\mathbf{x}\right|, 0, \dots, 0)^T$, используя преобразование Хаусхолдера.


In [3]:
def householder(vec):
    """Создает отражение Хаусхолдера, преобразующее 2-ю, 3-ю и т.д. компоненты вектора vec в нули.
    
    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Введенный вектор
    
    Returns
    -------
    outvec : array of floats, shape (n,)
        Преобразованный вектор, причем ``outvec[1:]==0`` и ``|outvec| == |vec|``
    H : array of floats, shape (n, n)
        Ортогональная матрица отражений Хаусхолдера
    """
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    else: 
        y = np.zeros(shape=(len(vec)))
        y[0] = np.linalg.norm(vec)
        u = (vec - y)/np.linalg.norm(vec - y)
        H = np.identity(len(vec)) - 2*u.reshape((len(vec), 1))*u
        return H.dot(vec), H

Протестируйте свою функцию на следующих примерах:

In [4]:
v = np.array([1, 2, 3])
v1, h = householder(v)

assert_allclose(h @ v1, v, atol=1e-14)
assert_allclose(h @ v, v1, atol=1e-14)

assert_allclose(v1[1:], 0, atol=1e-14)

assert_allclose(h @ h.T, np.eye(3), atol=1e-14)

###BEGIN HIDDEN TESTS
rndm = np.random.RandomState(1223)
vec = rndm.uniform(size=21)
v1, h = householder(vec)

assert_allclose(h @ v1, vec, atol=1e-14)
assert_allclose(h @ vec, v1, atol=1e-14)
assert_allclose(v1[1:], 0, atol=1e-14)

assert_allclose(h @ h.T, np.eye(h.shape[0]), atol=1e-14)
###END HIDDEN TESTS

### Part II. Вычисление $\mathrm{QR}$ - разложения матрицы.

Дана прямоугольная $m\times n$ матрица $\mathbf{A}$. Выполните отражение Хаусхолдера матрицы $\mathbf{H}_1$, преобразующее первый столбец матрицы $\mathbf{A}$ (назовем результат $\mathbf{A}^{(1)}$)

$$ 
\mathbf{H}_1 \mathbf{A} %\begin{pmatrix} \times &amp; \times &amp; \times &amp; \dots &amp; \times \\ 0 &amp; \times &amp; \times &amp; \dots &amp; \times \\ 0 &amp; \times &amp; \times &amp; \dots &amp; \times \\ &amp;&amp; \dots&amp;&amp; \\ 0 &amp; \times &amp; \times &amp; \dots &amp; \times \\ \end{pmatrix}
%
\equiv \mathbf{A}^{(1)}\;. 
$$

Теперь рассмотрим нижнюю правую часть матрицы $\mathbf{A}^{(1)}$ и выполним преобразование Хаусхолдера, действующее на 2 столбец $\mathbf{A}$:
$$
\mathbf{H}_2 \mathbf{A}^{(1)} % \begin{pmatrix} \times &amp; \times &amp; \times &amp; \dots &amp; \times \\ 0 &amp; \times &amp; \times &amp; \dots &amp; \times \\ 0 &amp; 0 &amp; \times &amp; \dots &amp; \times \\ &amp;&amp; \dots&amp;&amp; \\ 0 &amp; 0 &amp; \times &amp; \dots &amp; \times \\ \end{pmatrix}
%
\equiv \mathbf{A}^{(2)} \;. $$

Повторяя процесс $n$ раз, получим
$$
\mathbf{H}_{n} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;, 
$$

где $\mathbf{R}$ верхняя треугольная матрица. Так как каждая из матриц $\mathbf{H}_k$ ортогональна, таковым будет и их произведение. Обратная от ортогональной также есть матрица ортогональная. Таким образом, алгоритм создает $\mathrm{QR}$ - разложение матрицы $\mathbf{A}$.

Напишите функцию, получающую прямоугольную матрицу $A$ и возвращающую матрицы $Q$ и $R$ --- компоненты $QR$-разложения $A$.


In [5]:
def qr_decomp(a):
    """Вычисляет QR - разложение матрицы.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        Введенная матрица
    
    Returns
    -------
    q : ndarray, shape(m, m)
        Ортогональная матрица
    r : ndarray, shape(m, n)
        Верхняя треугольная матрица
        
    Examples
    --------
    >>> a = np.random.random(size=(3, 5))
    >>> q, r = qr_decomp(a)
    >>> np.testing.assert_allclose(q @ r, a, atol=1e-14)
    
    """
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    
    R = np.array(a, copy=True, dtype=float)
    Q = np.eye(m)
    
    
    for i in range(n):
        out, H = householder(R[i:,i])
        h = np.eye(m)
        h[i:,i:] = H
        
        R = h @ R
        Q = Q @ h
        
    return Q, R

In [6]:
# можете запустить данную операцию для бооее сжатого вывода: нули вместо `1e-16` и т.д.
np.set_printoptions(suppress=True)

In [7]:
rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
aa = a.copy()

q, r = qr_decomp(a)

# check that `qr_decomp` leaves `a` intact
assert_allclose(a, aa, atol=1e-16)

# тестируем, что Q ортогональна
assert_allclose(q @ q.T, np.eye(5), atol=1e-14)

# проверяем разложение
assert_allclose(q @ r, a, atol=1e-14)

Теперь сравните ваше разложение с разложением, полученным библиотечной функцией (содержащей соответствующие функции библиотеки LAPACK)

In [8]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a,
                atol=1e-14)

In [9]:
print(q, '\n', qq, '\n\n')
print(r, '\n', rr)

[[ 0.13665049  0.53601299 -0.09369752  0.7697136   0.30459557]
 [ 0.56035895  0.0935397  -0.53326881  0.01839528 -0.62652547]
 [ 0.19725922  0.65948912  0.60068463 -0.32384673 -0.24589462]
 [ 0.62498418 -0.50418303  0.52144688  0.28453698  0.04822969]
 [ 0.48765568  0.12171264 -0.27224305 -0.47049398  0.67223293]] 
 [[-0.13665049  0.53601299  0.09369752  0.661619   -0.49749149]
 [-0.56035895  0.0935397   0.53326881 -0.52477245 -0.34276292]
 [-0.19725922  0.65948912 -0.60068463 -0.37879015  0.14784752]
 [-0.62498418 -0.50418303 -0.52144688  0.18967657 -0.21750907]
 [-0.48765568  0.12171264  0.27224305  0.32774225  0.75222783]] 


[[ 1.40152769  1.2514379   0.89523615]
 [ 0.          0.84158252  0.68447942]
 [-0.          0.          0.5496046 ]
 [ 0.         -0.         -0.        ]
 [ 0.          0.          0.        ]] 
 [[-1.40152769 -1.2514379  -0.89523615]
 [ 0.          0.84158252  0.68447942]
 [ 0.          0.         -0.5496046 ]
 [ 0.          0.          0.        ]
 [ 0.    

Проверьте, согласуются ли ваши матрицы `q` и `r` с `qq` и `rr`. Объясните результат.
Оставьте пояснения в этой ячейке.
Матрицы совпадают с точностью до знака потому что при отражении получается вектор коллинеарный, поэтому и может возникать различие знаков. Выбор знака это выбор одного из двух возможных решений. 


### Часть III. Безматричная реализация.

Отметим необычную структуру матрицы Хаусхолдера: матрица поворота $\mathbf{H}$ полностью характеризуется вектором отражения $\mathbf{u}$. Заметим, также, что вычислительная сложность операции отражения матрицы сильно зависит от порядка операций:
$$ \left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A} \qquad \textrm{is } O(m^2 n)\;, $$

тогда как $$ \mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn) $$

Таким образом, следует избегать формирований матриц $\mathbf{H}$. Вместо этого можно сохранять сами векторы отражения $\mathbf{u}$ и производить умножение произвольной матрицы на $\mathbf{Q}^T$, например, как отдельную функцию (класс).

Напишите функцию, выполняющую QR - разложение матрицы без формирования матриц $\mathbf{H}$ и возвращающую матрицу $\mathbf{R}$, а также вектора отражений Хаусхолдера.


Напишите вторую функцию, которая использует вектора отражений, полученных предыдущей функцией, для вычисления произведения $Q^T B$ для заданной матрицы B. Убедитесь, что вы добавили достаточно комментариев, следующих вашим выкладкам. 



In [10]:
from math import copysign

def qr_nomatrix(a):
    """Form QR decomposition of `a` via successive Householder reflections.
    
    Parameters
    ----------
    a : ndarray
        Input matrix
    
    Returns
    -------
    R : ndarray
        Upper triangular matrix of the QR decomposition
    
    U : ndarray
        Columns store successive Householder reflectors: `U[j:, j]` stores
        the Householder reflector for reducing the `j`-th column.
        
    See Also
    --------
    householder_apply : apply Householder reflectors stored in `U` to `a`.
    
    """

    (num_rows, num_cols) = np.shape(a)

    # Initialize orthogonal matrix Q and upper triangular matrix R.
    Q = np.identity(num_rows)
    R = np.copy(a)

    # Iterative over column sub-vector and
    # compute Householder matrix to zero-out lower triangular matrix entries.
    for cnt in range(num_rows - 1):
        x = R[cnt:, cnt]

        e = np.zeros_like(x)
        e[0] = copysign(np.linalg.norm(x), -a[cnt, cnt])
        u = x + e
        v = u / np.linalg.norm(u)

        Q_cnt = np.identity(num_rows)
        Q_cnt[cnt:, cnt:] -= 2.0 * np.outer(v, v)

        R = Q_cnt @ R
        Q = Q @ Q_cnt.T

    return (R, Q)

    
def householder_apply(b, uu):
    """Apply the Householder reflectors stored in `uu` to `b`.
    
    The result is equivalent to
    >>> r, q = qr_decomp(a)
    >>> q.T @ b
    
    Parameters
    ----------
    b : ndarray
        Input matrix
    uu : ndarray
        Householder reflectors: `U[j:, j]` is the reflection vector
        for transforming the `j`-th column of `a`.
        
    Returns
    -------
    r : ndarray
        The result of applying the reflectors to `b`. Equivalent to
        computing `q.T @ b`.

    See Also
    --------
    qr_decomp
    
    """
    return  np.linalg.inv(uu) @ b
    
    

In [11]:
rndm = np.random.RandomState(1223)

a = rndm.uniform(size=(5, 5))
R1, U = qr_nomatrix(a)
R2 = householder_apply(a, U)
R_lib = qr(a)[1]
assert_allclose(R1, R2, atol=1e-14)

###BEGIN HIDDEN TESTS

name2 = "Test 2: vertical rectangular matrix"
a2 = rndm.uniform(size=(5, 3))
R1, U = qr_nomatrix(a2)
R22 = householder_apply(a2, U)
R_lib2 = qr(a2)[1]
assert_allclose(R1, R22, atol=1e-14)


name3 = "Test 3: horizontal rectangular matrix"
a3 = rndm.uniform(size=(3, 5))
R1, U = qr_nomatrix(a3)
R23 = householder_apply(a3, U)
R_lib3 = qr(a3)[1]
assert_allclose(R1, R23, atol=1e-14)

###END HIDDEN TESTS

IndexError: index 3 is out of bounds for axis 1 with size 3

In [12]:
# Check consistency with the scipy library decomposition. Allow for sign differences

conds = [np.allclose(R2[i, :], R_lib[i, :], atol=1e-14) or
         np.allclose(R2[i, :], -R_lib[i, :], atol=1e-14) for i in range(5)]
assert False not in conds

###BEGIN HIDDEN TESTS
conds = [np.allclose(R22[i, :], R_lib2[i, :], atol=1e-14) or
         np.allclose(R22[i, :], -R_lib2[i, :], atol=1e-14) for i in range(5)]
assert False not in conds

conds = [np.allclose(R23[i, :], R_lib3[i, :], atol=1e-14) or
         np.allclose(R23[i, :], -R_lib3[i, :], atol=1e-14) for i in range(3)]
assert False not in conds
###END HIDDEN TESTS

NameError: name 'R22' is not defined

In [13]:
# More testing here, keep this cell intact

### BEGIN HIDDEN TESTS

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))

q, r = qr_decomp(a)
rr, uu = qr_nomatrix(a)

# check consistency
assert_allclose(np.triu(r),
                np.triu(rr), atol=1e-14)

# $Q^T @ A = R$
assert_allclose(householder_apply(a, uu),
                r,
                atol=1e-14)

# act on a different matrix
b = rndm.uniform(size=(5, 3))
assert_allclose(householder_apply(b, uu),
                q.T @ b,
                atol=1e-14)

# END HIDDEN TESTS

IndexError: index 3 is out of bounds for axis 1 with size 3